In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install fasttext

     |████████████████████████████████| 71kB 9.3MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3038769 sha256=5536d54003d13df2ee074d4100f60284227ef35aed36e77fb12094883f61fe6f
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [3]:
import numpy as np
import logging
from gensim.models import Word2Vec
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import gensim
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
from gensim.models.keyedvectors import KeyedVectors
import fasttext
import fasttext.util

In [4]:
path = '/content/drive/MyDrive/competition_2/ner_hindi_train.txt'

In [5]:
read_file = pd.read_csv(path, header = None, error_bad_lines=False)

b'Skipping line 90: expected 1 fields, saw 2\nSkipping line 113: expected 1 fields, saw 2\nSkipping line 184: expected 1 fields, saw 2\nSkipping line 252: expected 1 fields, saw 2\nSkipping line 286: expected 1 fields, saw 2\nSkipping line 345: expected 1 fields, saw 2\nSkipping line 347: expected 1 fields, saw 2\nSkipping line 435: expected 1 fields, saw 2\nSkipping line 512: expected 1 fields, saw 2\nSkipping line 517: expected 1 fields, saw 2\nSkipping line 551: expected 1 fields, saw 2\nSkipping line 569: expected 1 fields, saw 2\nSkipping line 573: expected 1 fields, saw 2\nSkipping line 625: expected 1 fields, saw 2\nSkipping line 627: expected 1 fields, saw 2\nSkipping line 642: expected 1 fields, saw 2\nSkipping line 657: expected 1 fields, saw 2\nSkipping line 669: expected 1 fields, saw 2\nSkipping line 691: expected 1 fields, saw 2\nSkipping line 735: expected 1 fields, saw 2\nSkipping line 764: expected 1 fields, saw 2\nSkipping line 820: expected 1 fields, saw 2\nSkipping 

In [6]:
read_file[0]

0           लोकतांत्रिक\tother
1          विकेंद्रीकरण\tother
2                    की\tother
3                  दिशा\tother
4                   में\tother
                  ...         
1407098              का\tother
1407099              एक\tother
1407100         गांव\tlocation
1407101              है\tother
1407102               .\tother
Name: 0, Length: 1407103, dtype: object

In [7]:
words = []
tags = []
for line in read_file[0]:
    items = line.split()
    words.append(items[0])
    tags.append(items[1])

In [8]:
words = [words]
labels = set(tags)

In [9]:
words[0][1]

'विकेंद्रीकरण'

In [10]:
words[0][2]

'की'

In [11]:
path='/content/drive/MyDrive/cc.hi.300.bin'
model = fasttext.load_model(path)

In [12]:
m,n = np.shape(words)
X = []
for i in range(n):
    X.append(model.get_word_vector(words[0][i]))

In [13]:
le = preprocessing.LabelEncoder()
Y = le.fit_transform(tags)

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [15]:
print(np.shape(X_train),np.shape(Y_train))
print(np.shape(X_test), np.shape(Y_test))

(984972, 300) (984972,)
(422131, 300) (422131,)


In [16]:
clf = DecisionTreeClassifier(max_depth=5)

In [17]:
clf.fit(X_train, Y_train) 

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [18]:
Y_pred = clf.predict(X_test)

In [19]:
pred_labels = le.inverse_transform(Y_pred)

In [20]:
accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
print("Accuracy = {0:.2f}".format(accuracy))
print("Precision = {0:.2f}".format(precision))
print("Recall = {0:.2f}".format(recall))

Accuracy = 0.87
Precision = 0.85
Recall = 0.87


In [22]:
test = pd.read_csv('/content/drive/MyDrive/competition_2/ner_hindi_test .csv')

In [23]:
test = test.astype(str)

In [24]:
test.head()

,ID,Words
0,NER_0001,उनके
1,NER_0002,माता
2,NER_0003,-
3,NER_0004,पिता
4,NER_0005,तथा


In [25]:
tokens = test['Words']

In [26]:
tokens.head()

0    उनके
1    माता
2       -
3    पिता
4     तथा
Name: Words, dtype: object

In [27]:
test_data = []
for i in range(len(tokens)):
    test_data.append(model.get_word_vector(tokens[i]))

In [28]:
Yp = clf.predict(test_data)

In [30]:
test_data_pred = le.inverse_transform(Yp)

In [31]:
test_data_pred

array(['other', 'other', 'other', ..., 'other', 'other', 'other'],
      dtype='<U12')

In [32]:
test_data_pred.shape

(12665,)

In [33]:
test['Predicted labels'] = test_data_pred

In [34]:
test.drop(['Words'],axis = 1)

,ID,Predicted labels
0,NER_0001,other
1,NER_0002,other
2,NER_0003,other
3,NER_0004,other
4,NER_0005,other
...,...,...
12660,NER_12661,other
12661,NER_12662,other
12662,NER_12663,other
12663,NER_12664,other


In [35]:
test.to_csv('/content/drive/MyDrive/competition_2/pred_3.csv')